# Kredi Karti Dolandiricilik Tespiti
## Ogrenci: Mohamed Khaled Hajasaad - 22040301156

Bu notebook, kredi karti dolandiricilik tespiti icin iki farkli makine ogrenimi modeli uygular:
1. **Support Vector Machine (SVM - RBF Kernel)** - Margin tabanli model
2. **LightGBM** - Gelismis Boosting topluluk yontemi

Her iki model de hiperparametre ayari ve kapsamli degerlendirme metrikleri icerir.

## 1. Kutuphanelerin Yuklenmesi

In [ ]:
# Gerekli kutuphaneleri ice aktar
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, roc_curve, confusion_matrix, classification_report
)
from lightgbm import LGBMClassifier
import warnings
warnings.filterwarnings('ignore')

print("Kutuphaneler basariyla yuklendi!")

## 2. Temizlenmis Verilerin Yuklenmesi

In [ ]:
# Temizlenmis veri setini yukle
df = pd.read_csv('data/processed/creditcard_clean.csv')

print(f"Veri Seti Boyutu: {df.shape}")
print(f"\nSinif Dagilimi:")
print(df['Class'].value_counts())
print(f"\nDolandiricilik Orani: %{df['Class'].mean() * 100:.2f}")
df.head()

## 3. Veri Hazirlama ve StandardScaler
SVM mesafe/margin tabanli bir algoritma oldugu icin StandardScaler kullanmak zorunludur.

In [ ]:
# Ozellikler ve hedef degiskeni ayir
X = df.drop('Class', axis=1)
y = df['Class']

# Verileri egitim ve test setlerine bol
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# StandardScaler uygula - SVM icin zorunlu!
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"Egitim seti boyutu: {X_train.shape[0]}")
print(f"Test seti boyutu: {X_test.shape[0]}")
print(f"\nEgitim seti sinif dagilimi:")
print(y_train.value_counts())
print(f"\nStandardScaler basariyla uygulandi!")

## 4. Model 1: Support Vector Machine (SVM - RBF Kernel)

In [ ]:
# SVM parametre izgarasini tanimla
svm_param_grid = {
    'C': [0.1, 1, 10],
    'gamma': ['scale', 'auto', 0.1, 0.01],
    'kernel': ['rbf']
}

# SVM modelini baslat (probability=True ROC icin gerekli)
svm_model = SVC(probability=True, random_state=42, class_weight='balanced')

# RandomizedSearchCV uygula (SVM yavas oldugu icin)
svm_search = RandomizedSearchCV(
    svm_model,
    svm_param_grid,
    n_iter=8,
    cv=3,
    scoring='f1',
    n_jobs=-1,
    verbose=1,
    random_state=42
)

print("SVM (RBF Kernel) Hiperparametre Ayari ile Egitiliyor...")
print("NOT: StandardScaler kullanildi - margin tabanli algoritmalar icin zorunlu!")
svm_search.fit(X_train_scaled, y_train)

print(f"\nEn Iyi Parametreler: {svm_search.best_params_}")
print(f"En Iyi CV F1 Skoru: {svm_search.best_score_:.4f}")

In [ ]:
# En iyi SVM modelini al
best_svm = svm_search.best_estimator_

# Tahminleri yap
svm_pred = best_svm.predict(X_test_scaled)
svm_pred_proba = best_svm.predict_proba(X_test_scaled)[:, 1]

# Metrikleri hesapla
svm_accuracy = accuracy_score(y_test, svm_pred)
svm_precision = precision_score(y_test, svm_pred)
svm_recall = recall_score(y_test, svm_pred)
svm_f1 = f1_score(y_test, svm_pred)
svm_auc = roc_auc_score(y_test, svm_pred_proba)

print("=" * 50)
print("SVM (RBF KERNEL) SONUCLARI")
print("=" * 50)
print(f"Dogruluk (Accuracy):  {svm_accuracy:.4f}")
print(f"Kesinlik (Precision): {svm_precision:.4f}")
print(f"Duyarlilik (Recall):  {svm_recall:.4f}")
print(f"F1-Skoru:             {svm_f1:.4f}")
print(f"AUC:                  {svm_auc:.4f}")
print("\nSiniflandirma Raporu:")
print(classification_report(y_test, svm_pred, target_names=['Normal', 'Dolandiricilik']))

## 5. Model 2: LightGBM (Boosting Ensemble)

In [ ]:
# Dengesiz veri icin sinif agirligini hesapla
scale_pos_weight = len(y_train[y_train == 0]) / len(y_train[y_train == 1])

# LightGBM parametre izgarasini tanimla
lgbm_param_grid = {
    'num_leaves': [31, 50, 100],
    'learning_rate': [0.01, 0.05, 0.1],
    'n_estimators': [100, 200, 300],
    'max_depth': [5, 10, 15, -1]
}

# LightGBM modelini baslat
lgbm_model = LGBMClassifier(
    random_state=42,
    scale_pos_weight=scale_pos_weight,
    verbose=-1
)

# RandomizedSearchCV uygula
lgbm_search = RandomizedSearchCV(
    lgbm_model,
    lgbm_param_grid,
    n_iter=20,
    cv=3,
    scoring='f1',
    n_jobs=-1,
    verbose=1,
    random_state=42
)

print("LightGBM Hiperparametre Ayari ile Egitiliyor...")
lgbm_search.fit(X_train_scaled, y_train)

print(f"\nEn Iyi Parametreler: {lgbm_search.best_params_}")
print(f"En Iyi CV F1 Skoru: {lgbm_search.best_score_:.4f}")

In [ ]:
# En iyi LightGBM modelini al
best_lgbm = lgbm_search.best_estimator_

# Tahminleri yap
lgbm_pred = best_lgbm.predict(X_test_scaled)
lgbm_pred_proba = best_lgbm.predict_proba(X_test_scaled)[:, 1]

# Metrikleri hesapla
lgbm_accuracy = accuracy_score(y_test, lgbm_pred)
lgbm_precision = precision_score(y_test, lgbm_pred)
lgbm_recall = recall_score(y_test, lgbm_pred)
lgbm_f1 = f1_score(y_test, lgbm_pred)
lgbm_auc = roc_auc_score(y_test, lgbm_pred_proba)

print("=" * 50)
print("LIGHTGBM SONUCLARI")
print("=" * 50)
print(f"Dogruluk (Accuracy):  {lgbm_accuracy:.4f}")
print(f"Kesinlik (Precision): {lgbm_precision:.4f}")
print(f"Duyarlilik (Recall):  {lgbm_recall:.4f}")
print(f"F1-Skoru:             {lgbm_f1:.4f}")
print(f"AUC:                  {lgbm_auc:.4f}")
print("\nSiniflandirma Raporu:")
print(classification_report(y_test, lgbm_pred, target_names=['Normal', 'Dolandiricilik']))

## 6. ROC Egrileri Karsilastirmasi

In [ ]:
# ROC egrilerini hesapla
svm_fpr, svm_tpr, _ = roc_curve(y_test, svm_pred_proba)
lgbm_fpr, lgbm_tpr, _ = roc_curve(y_test, lgbm_pred_proba)

# ROC egrilerini ciz
plt.figure(figsize=(10, 8))

plt.plot(svm_fpr, svm_tpr, label=f'SVM (AUC = {svm_auc:.4f})', linewidth=2)
plt.plot(lgbm_fpr, lgbm_tpr, label=f'LightGBM (AUC = {lgbm_auc:.4f})', linewidth=2)
plt.plot([0, 1], [0, 1], 'k--', label='Rastgele Siniflandirici', linewidth=1)

plt.xlabel('Yanlis Pozitif Orani (FPR)', fontsize=12)
plt.ylabel('Dogru Pozitif Orani (TPR)', fontsize=12)
plt.title('ROC Egrisi Karsilastirmasi', fontsize=14)
plt.legend(loc='lower right', fontsize=11)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('roc_curves.png', dpi=300, bbox_inches='tight')
plt.show()

print("ROC Egrisi 'roc_curves.png' olarak kaydedildi")

## 7. Karisiklik Matrisleri

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# SVM Karisiklik Matrisi
svm_cm = confusion_matrix(y_test, svm_pred)
sns.heatmap(svm_cm, annot=True, fmt='d', cmap='Blues', ax=axes[0],
            xticklabels=['Normal', 'Dolandiricilik'],
            yticklabels=['Normal', 'Dolandiricilik'])
axes[0].set_title('SVM - Karisiklik Matrisi', fontsize=12)
axes[0].set_xlabel('Tahmin Edilen')
axes[0].set_ylabel('Gercek')

# LightGBM Karisiklik Matrisi
lgbm_cm = confusion_matrix(y_test, lgbm_pred)
sns.heatmap(lgbm_cm, annot=True, fmt='d', cmap='Greens', ax=axes[1],
            xticklabels=['Normal', 'Dolandiricilik'],
            yticklabels=['Normal', 'Dolandiricilik'])
axes[1].set_title('LightGBM - Karisiklik Matrisi', fontsize=12)
axes[1].set_xlabel('Tahmin Edilen')
axes[1].set_ylabel('Gercek')

plt.tight_layout()
plt.savefig('confusion_matrices.png', dpi=300, bbox_inches='tight')
plt.show()

## 8. Sonuc Karsilastirma Tablosu

In [ ]:
# Sonuc DataFrame olustur
results_df = pd.DataFrame({
    'Model': ['SVM (RBF)', 'LightGBM'],
    'Dogruluk': [svm_accuracy, lgbm_accuracy],
    'Kesinlik': [svm_precision, lgbm_precision],
    'Duyarlilik': [svm_recall, lgbm_recall],
    'F1-Skoru': [svm_f1, lgbm_f1],
    'AUC': [svm_auc, lgbm_auc]
})

# Degerleri yuvarla
results_df = results_df.round(4)

# Sonuclari goster
print("=" * 70)
print("MODEL KARSILASTIRMA SONUCLARI")
print("=" * 70)
print(results_df.to_string(index=False))

# Sonuclari CSV dosyasina kaydet
results_df.to_csv('model_comparison_results.csv', index=False)
print("\nSonuclar 'model_comparison_results.csv' dosyasina kaydedildi")

## 9. En Iyi Model Analizi

In [ ]:
# F1-Skoruna gore en iyi modeli belirle
best_model_idx = results_df['F1-Skoru'].idxmax()
best_model_name = results_df.loc[best_model_idx, 'Model']

print("=" * 70)
print("SON ANALIZ")
print("=" * 70)
print(f"\nEN IYI MODEL: {best_model_name}")
print(f"\nEn Iyi Model Metrikleri:")
print(results_df.loc[best_model_idx].to_string())

print("\n" + "-" * 70)
print("ANALIZ NOTLARI:")
print("-" * 70)
print("""
1. SVM ve LightGBM Karsilastirmasi:
   - SVM: Margin tabanli, yuksek boyutlu verilerde etkili
   - LightGBM: Hizli ve verimli gradient boosting algoritmasi

2. StandardScaler Etkisi:
   - SVM icin olcekleme kritik oneme sahip (RBF kernel mesafe hesaplar)
   - LightGBM agac tabanli oldugu icin olceklemeye duyarsiz

3. Dengesiz Veri Seti:
   - Dolandiricilik vakalari cok nadir (%0.17)
   - SVM'de class_weight='balanced' kullanildi
   - LightGBM'de scale_pos_weight kullanildi
   - F1-Skoru ve AUC daha guvenilir metriklerdir

4. Asiri Ogrenme/Yetersiz Ogrenme:
   - SVM: C parametresi cok yuksekse asiri ogrenme riski
   - LightGBM: num_leaves ve max_depth ile kontrol edilir
   - Capraz dogrulama ile kontrol edildi

5. Oneriler:
   - LightGBM genellikle daha hizli ve olceklenebilir
   - SVM buyuk veri setlerinde yavas olabilir
   - Dolandiricilik tespitinde Duyarlilik (Recall) onemli
""")

## 10. Ozellik Onemi (LightGBM)

In [ ]:
# LightGBM ozellik onemini gorsellestir
plt.figure(figsize=(12, 8))

lgbm_importance = pd.DataFrame({
    'Ozellik': X.columns,
    'Onem': best_lgbm.feature_importances_
}).sort_values('Onem', ascending=True).tail(15)

plt.barh(lgbm_importance['Ozellik'], lgbm_importance['Onem'], color='forestgreen')
plt.title('LightGBM - En Onemli 15 Ozellik', fontsize=14)
plt.xlabel('Onem Derecesi')
plt.tight_layout()
plt.savefig('feature_importance.png', dpi=300, bbox_inches='tight')
plt.show()

---
## Ozet

Bu notebook sunlari uyguladi:
1. Support Vector Machine (SVM - RBF Kernel) - Hiperparametre Ayarli (C, gamma)
2. LightGBM (Boosting Ensemble) - Hiperparametre Ayarli (num_leaves, learning_rate, n_estimators, max_depth)
3. StandardScaler - Margin tabanli algoritmalar icin zorunlu
4. Tam degerlendirme metrikleri (Dogruluk, Kesinlik, Duyarlilik, F1-Skoru, AUC)
5. Her iki model icin ROC Egrileri
6. Karisiklik Matrisleri
7. Tum metriklerle karsilastirma tablosu
8. LightGBM ozellik onemi analizi

**Veri Kaynagi:** `data/processed/creditcard_clean.csv` (Data_Cleaning.ipynb dosyasinda on islenmis)